In [1]:
## sudo apt install default-jdk 터미널에 실행해주세영
from konlpy.tag import Komoran
from konlpy.utils import pprint
from collections import Counter
import re
import pandas as pd

LAB2NUM = {'IT과학': 0,
         '경제': 1,
         '사회': 2,
         '생활문화': 3,
         '세계': 4,
         '스포츠': 5,
         '정치': 6,
         }

NUM2LAB = {0 : 'IT과학',
           1 : '경제',
           2 : '사회',
           3 : '생활문화',
           4 : '세계',
           5 : '스포츠',
           6 : '정치'}


# Initialize Komoran for Korean morphological analysis
komoran = Komoran()



In [11]:
def get_predefined_label(train_data):
    """
    설명 : url column으로 언론사에서 자체 분류한 target을 구하는 함수
    parameters : 기존 train data(pd.DataFrame)
    return : predefined 컬럼이 추가 된 데이터(pd.DataFrame)
    """
    naver = {'105' : 'IT과학',
         '104' : '세계',
         '103' : '생활문화',
         '102' : '사회',
         '101' : '경제',
         '100' : '정치'
         }

    pre_target = []
    for url in train_data['url']:
        if 'sports' in url:
            pre_target.append(LAB2NUM['스포츠'])
        else:
            p = re.compile(r'sid1=10[0-5]')
            num = p.findall(url)[0][5:8]
            pre_target.append(LAB2NUM[naver[num]])

    train_data['predefined'] = pre_target

    return train_data

# Define a function to extract keywords from text
def extract_keywords(text):
    '''
    설명 : text(str type)를 넣으면 형태소 분석을 해서 명사, 용언, 수식언만 추출하는 함수
    input : text(str type)
    return : keywords(list type)
    '''
    # Tokenize the text using Komoran
    tokens = komoran.pos(text)
    
    # Define the POS tags for nouns, verbs, and modifiers
    pos_tags = ['NNG', 'NNP', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VX', 'MM', 'MAG', 'MAJ']
    keywords = [token[0] for token in tokens if token[1] in pos_tags]
    
    return keywords

def get_keyword_of_target(target, train_data):
    '''
    설명 : 입력한 target의 keyword와 빈도수 추출
    input
    - target : 키워드를 추출할 target(str)
    - train_data : 입력 데이터 (pd.DataFrame)
    return : ['keyword','count'] column의 output(pd.DataFrame)
    '''
    target = LAB2NUM[target]
    target_data = train_data[train_data['target'] == target]
    target_keywords = []
    for text in target_data['text']:
        keywords = extract_keywords(text)
        target_keywords.extend(keywords)
    frequency = Counter(target_keywords)
    sorted_keywords = sorted(frequency.items(), key=lambda x: x[1], reverse=True)
    output = pd.DataFrame(sorted_keywords[:500], columns=['keyword','count'])
    return output


def compare_keywords(target, train_data):
    '''
    설명 : target과 predefined를 비교해서 같을 때와 다를 때의 keyword를 비교하는 함수
    input : 비교할 target(str)
    return
    - same : target과 predefined가 같은 text data의 keyword(pd.DataFrame)
    - diff : target과 predefined가 다른 text data의 keyword(pd.DataFrame)
    '''
    t = LAB2NUM[target]
    data = train_data[train_data['predefined'] == t]
    same = get_keyword_of_target(target, data)
    data = train_data[train_data['predefined'] != t]
    diff = get_keyword_of_target(target, data)
    output = pd.concat([same, diff], axis=1)
    return output

def text_examples(keyword, target, data):
    idx = []
    t = LAB2NUM[target]
    d = data[(data['predefined'] != t) & (data['target'] == target)]
    for i in range(len(d)):
        if keyword in d.iloc[i]['text']:
            print(d.iloc)
            idx.append(i)
    output = data.iloc[idx]
    return output


In [64]:
train = pd.read_csv('../data/no_noise_hanja2hangeull.csv')

In [ ]:
k = []
for i in range(len(train)):
  a=re.findall('[一-龥]', train['text'].iloc[i])
  if len(a)!=0:
    k=[*k,*a]

Counter(k).most_common()[:35]

In [51]:
d = { "中": "중국", "美": "미국","北":"북한",'日':"일본",'英':'영국','行':'행','靑':'청와대','朴':'박','銀':'은행','與':'여당',
     '文':'문','野':'야당','獨':'독일','伊':'이탈리아','韓':'한국','佛':'프랑스','前':'전','檢':'검찰','軍':'군','安':'안철수','南':'남한',
     '亞':'아시아','展':'전시회','重':'차','株':'주식','詩':'시', '反': '반대'}

In [60]:
for key, val in d.items():
    train['text'] = train['text'].str.replace(key, val)

train[train['target'] == 4].head(10)

,Unnamed: 0,ID,text,target,url,date
8,8,ynat-v1_train_00008,왓츠앱稅 230원에 성난 레바논 민심…총리사퇴로 이어져종합2보,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.30. 오전 4:17
9,9,ynat-v1_train_00009,베트남 경제 고성장 지속…2분기 GDP 6.71% 성장,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.28. 오후 5:42
10,10,ynat-v1_train_00010,그리스서 한국전 참전 기념식…참전용사 한반도 평화 기원,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.26. 오전 1:49
12,12,ynat-v1_train_00012,미국 베네수엘라 구호품 반입 촉구 안보리 결의 추진,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.27. 오전 8:21
18,18,ynat-v1_train_00018,파키스탄 경제난 속 카타르서 30억 달러 투자 유치,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.25. 오후 12:55
19,19,ynat-v1_train_00019,일본 예산팽창에 재정건전성 우려…내년에 장기채무 GDP 2배,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.21. 오후 2:22
20,20,ynat-v1_train_00020,칠레 민중가수 살해하고 미국 도피 전 군인 법정에 선다,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.03. 오전 6:20
21,21,ynat-v1_train_00021,일본 온라인으로 중국인 비자신청 받는다,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.10. 오전 10:50
22,22,ynat-v1_train_00022,미 동부 마비시킨 악성 파일 국내서도 발견,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.01.25. 오후 2:13
32,32,ynat-v1_train_00032,일본경제산업상 경제보복 비판여론에 언론이 제대로 이해못해,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.04. 오후 5:15


In [63]:
train = train[['ID', 'text', 'target', 'url', 'date']]
train.to_csv("../data/no_noise_hanja2hangeull.csv", index=False)

In [66]:
target = 'IT과학'
train = get_predefined_label(train)
#output = get_keyword_of_target(target, train):
output = compare_keywords(target, train)
print(f'{target} 키워드 비교')
print(output.head(10))

IT과학 키워드 비교
  keyword  count keyword  count
0      출시    559      종합     25
1      종합    380      로봇     24
2      개발    350      개발     19
3     갤럭시    282      미국     18
4      기술    278      북한     15
5       폰    272      중국     15
6     서비스    241      기술     14
7       개    239       억     12
8       만    235      시장     12
9     네이버    230    LG전자     11


In [67]:
for key in LAB2NUM.keys():
    output = compare_keywords(key, train)
    print(f'{key} 키워드 비교')
    print(output.head(10)) 

IT과학 키워드 비교
  keyword  count keyword  count
0      출시    559      종합     25
1      종합    380      로봇     24
2      개발    350      개발     19
3     갤럭시    282      미국     18
4      기술    278      북한     15
5       폰    272      중국     15
6     서비스    241      기술     14
7       개    239       억     12
8       만    235      시장     12
9     네이버    230    LG전자     11
경제 키워드 비교
  keyword  count keyword  count
0       억    711      종합    162
1      종합    490       억    144
2      분기    429      출시    119
3      영업    365      분기    111
4      투자    358       조     94
5       주    340       만     86
6      금융    336       천     83
7      코스    279      투자     80
8       피    279      삼성     74
9       천    271      요금     73
사회 키워드 비교
  keyword  count keyword  count
0      방송    139      종합    326
1      언론    131     게시판    175
2      종합     99       만    107
3      사장     99      한국    104
4      기자     94      지원     85
5      노조     75       억     83
6     게시판     70      미래     73
7      방

In [33]:
def text_examples(keyword, target, data):
    idx = []
    t = LAB2NUM[target]
    d = data[data['predefined'] != t]
    d = d[d['target'] == t]
    for i in range(len(d)):
        #print(d.iloc[i]['text'])
        if keyword in d.iloc[i]['text']:
            print(d.iloc[i]['text'])
            idx.append(i)
    output = data.iloc[idx]
    return output


In [34]:
text = text_examples('올림픽', '스포츠', train)

올림픽 7시간 손에 쥔 한반도기…조선총련 응원단 속초 도착
올림픽 1번 주자는 전이경
올림픽 아침을 여는 북한 응원단
평창동계올림픽 G1년…세계 시선 평창 집중
올림픽 문화올림픽 잔칫날 3일 개막…세계인 화합 한마당
아르헨 올림픽 축구대표팀 멕시코서 금품 도난당해
평창올림픽 G1년…KT 5G 기반으로 플랫폼 사업자 변신종합
올림픽 문화올림픽 합동 세배 등 행사 다채…세계인 눈길
리우올림픽 출전 선수 격려하는 조양호 회장
올림픽 평화를 선물한 다서 다이 남부 깁짱 때 막 싸울까 부란해써요
올림픽 입항하는 만경봉호
올림픽 기록 확인하는 북한 최명광
북한 선수 22명 평창올림픽 참가…올림픽 첫 단일팀 확정종합
올림픽 슬로프 내려오는 북한 최명광
올림픽 날씨 쇼트트랙 女1500m·男1000m 결승…강릉 9∼3도
올림픽 경기 마친 최명광
국회 도쿄올림픽 경기장 욱일기 반입금지 촉구 결의안 의결종합
토리노 동계올림픽 공동입장하는 남북
리우올림픽 개막식에 2명의 대통령…직무정지 호세프 참석 희망
누워서 타는 자전거로 평창동계올림픽 홍보
동계올림픽기간 강원도 외국인 신용카드 지출 3배로
중국 선양서 평창올림픽 홍보하는 주니어 피겨대회 열려
올림픽 평창올림픽 썽공기원 군뉴스코 페스티벌
올림픽 쓰케이트 나레 수믄 과학…얄꼬 길고·두껍꼬 짤꼬
외교관·외신기자 100여명 평창올림픽 ICT체험관서 탄성종합
KT 평창동계올림픽 준비현장 점검
KIST 도핑컨트롤센터 도핑 없는 평창 동계올림픽 준비 중
올림픽 재정난 가나팀…韓기업가 후원받고 평창으로종합
올림픽 팀코리아 경기 보러가자
올림픽 한반도기 게양한 만경봉호
올림픽 와우 개회식에서 터진 다섯 번의 탄성
올림픽 팀 코리아 감독은 나
올림픽 문화로 물들이다…강원국제비엔날레 내달 개막
올림픽 인공기 흔드는 북한 응원단
올림픽 마이크 펜스 뒤로 김영남과 김여정
올림픽 김정숙 여사 간절한 마음
SNS돋보기 볼거리 풍성 평창올림픽 개막식…상상 이상
미래부 평창 동계올림픽 D2년 특별세미나 개최
관광올림픽에 승부…강원도 외국인 관광객 300만명

In [32]:
text

,Unnamed: 0,ID,text,target,url,date,predefined
3,3,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54,0
13,13,ynat-v1_train_00013,황재균 쐐기타…kt 갈 길 바쁜 삼성에 고춧가루,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.09.16 17:28,5
25,25,ynat-v1_train_00025,예스24 독자들이 선정한 올해의 책 여행의 이유,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.10. 오전 11:01,3
37,37,ynat-v1_train_00037,베네수엘라 대통령 미국과 관계 복원 대화 재개 합의,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.23. 오전 6:26,4
42,42,ynat-v1_train_00042,제임스 떠난 NBA 클리블랜드 1승 10패…전체 꼴찌,4,https://sports.news.naver.com/news.nhn?oid=001...,2018.11.08 15:06,5
...,...,...,...,...,...,...,...
510,510,ynat-v1_train_00510,현대차증권 2분기 영업이익 414억원…232% 증가종합,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.25. 오전 10:23,1
515,515,ynat-v1_train_00515,5G 시대 스마트제조혁신과 융합보안 세미나,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.08. 오후 1:53,0
518,518,ynat-v1_train_00518,홍영표 일자리 으뜸기업 방문…고용창출 격려,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.06. 오전 5:00,6
520,520,ynat-v1_train_00520,美中무역전쟁 해결 기대에 원달러 16원↓…22개월만에 최대낙폭종합,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.11.02. 오후 4:21,1


In [68]:
train['text'] = train['text'].str.replace('게시판', ' ')
train['text'] = train['text'].str.replace('종합', ' ')
train['text'] = train['text'].str.replace('북한', ' ')
train['text'] = train['text'].str.replace('  ', ' ')
train.to_csv("../data/no_noise_hanja2kor_delkey.csv", index=False)

# 공공데이터 뉴스 전처리

In [5]:
data = pd.read_csv('../data/public_news_data.csv')
data.describe()

,일자,제목,통합 분류1,통합 분류2,통합 분류3
count,1067264,1067252,1026137,864587,667254
unique,5551,905703,18,16,9
top,2021-07-27,동정,정치,정치,지역
freq,4642,1046,255995,226030,155193


In [6]:
data = data.drop_duplicates(['제목'], keep='first')

In [9]:
labels = ['경제', '사회', 'IT', '문화', '정치', '국제', '스포']

In [15]:
data.describe()

,일자,제목,통합 분류1,통합 분류2,통합 분류3
count,905704,905703,870193,730610,563309
unique,5537,905703,18,16,9
top,2021-07-27,"성동구, 중장년 1인가구 안심방역 박스 지원",정치,정치,지역
freq,3851,1,192667,168997,135285


In [16]:
data.index

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       1067341, 1067342, 1067343, 1067344, 1067345, 1067346, 1067348, 1067349,
       1067350, 1067351],
      dtype='int64', length=905704)

In [23]:
notlabels = []
for i in range(len(data)):
    if (data['통합 분류1'].iloc[i] in labels) or (data['통합 분류2'].iloc[i] in labels) or (data['통합 분류3'].iloc[i] in labels):
        notlabels.append(True)
    else:
        notlabels.append(False)
data['label'] = notlabels
print(notlabels.sum())
data.head()

AttributeError: 'list' object has no attribute 'sum'

In [27]:
data[data['label']==False].index

Index([      0,       1,       7,      15,      17,      20,      26,      28,
            31,      33,
       ...
       1067302, 1067308, 1067309, 1067326, 1067336, 1067338, 1067341, 1067343,
       1067345, 1067349],
      dtype='int64', length=129824)

In [28]:
notlabel = data[data['label']==False].index
data = data.drop(notlabel)
data.describe()

,일자,제목,통합 분류1,통합 분류2,통합 분류3,label
count,775880,775880,775880,641167,476809,775880
unique,5482,775880,10,9,9,1
top,2021-07-27,"[글로벌혁신기술대상] 석승국 ㈜재일원(크린파파) 대표 ""'눈엣가시' 키우지 말고 직...",정치,정치,정치,True
freq,3211,1,192667,168997,109636,775880


In [33]:
import numpy as np

In [48]:
data = data.replace('00', np.NaN)
data = data.replace('99', np.NaN)

In [50]:
data['통합 분류1'].unique()

array(['경제', '사회', '지역', 'IT', '문화', '정치', '국제', '스포', nan], dtype=object)

In [53]:
data[data['통합 분류1'].isna() & data['통합 분류2'].isna()  & data['통합 분류3'].isna()]

,일자,제목,통합 분류1,통합 분류2,통합 분류3,label


In [56]:
first = data[data['통합 분류1'].notna() & data['통합 분류2'].isna() & data['통합 분류3'].isna()]
first['label'] = first['통합 분류1']
first['text'] = first['제목']
first = first[['text', 'label']]
first.to_csv('../data/first.csv', index=False)

/tmp/ipykernel_25312/4023667266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first['label'] = first['통합 분류1']
/tmp/ipykernel_25312/4023667266.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first['text'] = first['제목']


In [58]:
data = data.drop(first.index)
data.describe()

,일자,제목,통합 분류1,통합 분류2,통합 분류3,label
count,641166,641166,641151,641166,476808,641166
unique,5382,641166,8,8,8,1
top,2021-07-27,"[글로벌혁신기술대상] 석승국 ㈜재일원(크린파파) 대표 ""'눈엣가시' 키우지 말고 직...",정치,정치,정치,True
freq,2726,1,166767,168997,109636,641166


In [61]:
data[(data['통합 분류1'] == data['통합 분류2'])]

,일자,제목,통합 분류1,통합 분류2,통합 분류3,label
3,2021-12-31,"[신년사] 정영애 여가부 장관 ""포용 평등 정신으로 취약계층 살필 것""",사회,사회,NaN,True
6,2021-12-31,"정영애 여가부장관 ""젠더폭력 철저히 대응 일터에선 성평등 촉진""",사회,사회,정치,True
8,2021-12-31,코로나19시대도 서울시 자치구 홍보맨 전성시대!,사회,사회,사회,True
9,2021-12-31,"관악구, ‘EVERY 스마트도시 관악’조성 마스터플랜 수립",지역,지역,IT,True
10,2021-12-31,"3억이 50억으로 공무원 출신 파이어족 ""집살때 3가지 꼭 보세요""",경제,경제,경제,True
...,...,...,...,...,...,...
1067329,2017-06-30,스타트업 노하우 전수…기보 ‘제1회 벤처포럼’,경제,경제,경제,True
1067330,2017-06-30,"진웅섭 ""카드업계 전망 어두워…건전성 지표 관리 필요""",경제,경제,경제,True
1067332,2017-06-30,"""中 뛰는데 우리는 규제프리존 논의조차 못해""",정치,정치,NaN,True
1067339,2017-06-30,한국광산업진흥회 ‘2017 국제광융합엑스포’ 성황리 마무리,경제,경제,IT,True


In [62]:
onetwo = data[(data['통합 분류1'] == data['통합 분류2'])]
onetwo['label'] = onetwo['통합 분류1']
onetwo['text'] = onetwo['제목']
onetwo = onetwo[['text', 'label']]
onetwo.to_csv('../data/onetwo.csv', index=False)
data = data.drop(onetwo.index)
data.describe()

/tmp/ipykernel_25312/1952441613.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onetwo['label'] = onetwo['통합 분류1']
/tmp/ipykernel_25312/1952441613.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onetwo['text'] = onetwo['제목']


,일자,제목,통합 분류1,통합 분류2,통합 분류3,label
count,206461,206461,206446,206461,154000,206461
unique,4812,206461,8,8,8,1
top,2021-07-15,"[글로벌혁신기술대상] 석승국 ㈜재일원(크린파파) 대표 ""'눈엣가시' 키우지 말고 직...",사회,국제,지역,True
freq,1004,1,45282,36437,42166,206461


In [64]:
data[(data['통합 분류2'] == data['통합 분류3'])]

,일자,제목,통합 분류1,통합 분류2,통합 분류3,label
16,2021-12-30,"자율순찰로봇, AI관제 시스템 관악구 스마트도시 사업 박차",IT,지역,지역,True
21,2021-12-30,"청주시, 주택정책 기본계획 방향 논의 1인가구 비중 34→37.5% 증가",경제,지역,지역,True
24,2021-12-29,"주택공급 규칙 개정, 내집마련 가능성 올라간 청포족 생애최초 및 1인가구가 눈 여겨...",경제,지역,지역,True
37,2021-12-29,"병원가기 막막했는데 오세훈표 안심동행서비스 96% ""매우 만족""",사회,IT,IT,True
52,2021-12-27,세종지역 청년 3명 중 2명이 1인가구 33.9% 전 월세 거주,사회,경제,경제,True
...,...,...,...,...,...,...
1067215,2017-06-29,음성지역발전이라는 주제로 산학관 머리 맞대,지역,경제,경제,True
1067226,2017-06-29,"[모닝브리핑] 대신증권 “로봇, 아시아 중심 수요 증가 전망”",IT,경제,경제,True
1067232,2017-06-30,창업 후 농업벤처 미국진출 돕는다,IT,경제,경제,True
1067243,2017-06-30,기보 '대학 기술 사업화' 9000억 지원,IT,경제,경제,True


In [65]:
tt = data[(data['통합 분류2'] == data['통합 분류3'])]
tt['label'] = tt['통합 분류2']
tt['text'] = tt['제목']
tt = tt[['text', 'label']]
tt.to_csv('../data/tt.csv', index=False)
data = data.drop(tt.index)
data.describe()

/tmp/ipykernel_25312/3606708109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt['label'] = tt['통합 분류2']
/tmp/ipykernel_25312/3606708109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tt['text'] = tt['제목']


,일자,제목,통합 분류1,통합 분류2,통합 분류3,label
count,129168,129168,129160,129168,76707,129168
unique,4657,129168,8,8,8,1
top,2021-07-15,"[글로벌혁신기술대상] 석승국 ㈜재일원(크린파파) 대표 ""'눈엣가시' 키우지 말고 직...",경제,사회,지역,True
freq,613,1,23969,25415,15225,129168


In [67]:
ot = data[(data['통합 분류1'] == data['통합 분류3'])]
ot['label'] = ot['통합 분류1']
ot['text'] = ot['제목']
ot = ot[['text', 'label']]
ot.to_csv('../data/ot.csv', index=False)
data = data.drop(ot.index)
data.describe()

/tmp/ipykernel_25312/1492357966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ot['label'] = ot['통합 분류1']
/tmp/ipykernel_25312/1492357966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ot['text'] = ot['제목']


,일자,제목,통합 분류1,통합 분류2,통합 분류3,label
count,63691,63691,63683,63691,11230,63691
unique,4308,63691,8,8,8,1
top,2021-07-14,“혼자 병원갈 때 ‘안심동행’ 부르세요”,사회,정치,국제,True
freq,305,1,15764,12522,1933,63691


In [69]:
total = pd.concat([first, onetwo, tt, ot])
total.describe()

,text,label
count,712189,712189
unique,712189,8
top,"[신년사]정영애 ""코로나19로 양극화 심화, 포용 공존이 살 길""",정치
freq,1,181662


In [71]:
total = total.replace(r'"','', regex=True)
total.head()

,text,label
4,"[신년사]정영애 코로나19로 양극화 심화, 포용 공존이 살 길",사회
12,골방고시원서 세밑한파 속 잇단 ‘외로운 죽음’,사회
13,스마트도시 체계적 조성 관악구 5개년 마스터플랜,IT
22,수돗물 안나오고 화장실은 얼고 우리한테 난방은 사치 [현장르포],사회
23,주택공급 규칙 개정...청포족 위한 내 집 마련 가능 단지는,경제


In [ ]:
LAB2NUM = {'IT과학': 0,
         '경제': 1,
         '사회': 2,
         '생활문화': 3,
         '세계': 4,
         '스포츠': 5,
         '정치': 6,
         }

In [3]:
total['label'].unique()

array(['사회', 'IT', '경제', '정치', '문화', '국제', '스포', '지역'], dtype=object)

In [4]:
idx = total[total['label'] == '지역'].index

In [5]:
total = total.drop(idx)
total.describe()

,text,label
count,562392,562392
unique,562392,7
top,‘경찰만 믿고 살기엔 ’ 2030 여성들 각자도생 방범대책,정치
freq,1,151199


In [6]:
total['label'].unique()

array(['사회', '경제', 'IT', '정치', '문화', '국제', '스포'], dtype=object)

In [2]:
a = pd.read_csv('../data/first.csv')
b= pd.read_csv('../data/onetwo.csv')
c = pd.read_csv('../data/ot.csv')
d = pd.read_csv('../data/tt.csv')
total = pd.concat([a,b,c,d])

In [ ]:
'사회', '경제', 'IT', '정치', '문화', '국제', '스포'

In [7]:
total['label'] = total['label'].replace('IT', 0)
total['label'] = total['label'].replace('경제', 1)
total['label'] = total['label'].replace('사회', 2)
total['label'] = total['label'].replace('문화', 3)
total['label'] = total['label'].replace('국제', 4)
total['label'] = total['label'].replace('스포', 5)
total['label'] = total['label'].replace('정치', 6)

In [ ]:
total['label']

In [9]:
total['target'] = total['label']
total = total[['text','target']]
total.head()

,text,target
8,‘경찰만 믿고 살기엔 ’ 2030 여성들 각자도생 방범대책,2
10,전북지역 1인 가구 67.7% 자기집 거주,2
11,"""4인가구에 맞춰진 주택정책 확 바꿔야""",1
12,"‘효성해링턴 플레이스 안성 공도’ 신혼부부, 무주택자 대상 특별공급 추첨",1
13,'이웃하다' 등 여성가족형 예비사회적 기업 22곳 지정,2


In [11]:
total = total.replace(r'"','', regex=True)
total.head()

,text,target
8,‘경찰만 믿고 살기엔 ’ 2030 여성들 각자도생 방범대책,2
10,전북지역 1인 가구 67.7% 자기집 거주,2
11,4인가구에 맞춰진 주택정책 확 바꿔야,1
12,"‘효성해링턴 플레이스 안성 공도’ 신혼부부, 무주택자 대상 특별공급 추첨",1
13,'이웃하다' 등 여성가족형 예비사회적 기업 22곳 지정,2


In [13]:
total.to_csv("../data/public_data.csv", index=False)

In [9]:
a = pd.read_csv('../data/no_noise_data.csv')
b = pd.concat([a, df])
b.head()

,Unnamed: 0,ID,text,target,url,date
0,0.0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.06.30. 오전 10:36
1,1.0,ynat-v1_train_00001,어버이날 맑다가 흐려져 남부 지방 옅은 황사,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.08. 오전 5:25
2,2.0,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.03.15. 오후 12:00
3,3.0,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.28. 오전 9:54
4,4.0,ynat-v1_train_00004,회색 인간 작가 김동식 양심고백 등 새 소설집 2권 출간,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.04.03. 오전 7:05


In [10]:
b = b[['ID', 'text', 'target', 'url', 'date']]

In [12]:
len(b)

115678

In [19]:
b.to_csv('../data/last.csv', index=False)

In [2]:
data = pd.read_csv("../data/public_data.csv")
len(data)

562392

In [4]:
again = pd.DataFrame()
again = pd.concat([data[data['target'] == 0][:10000],
                   data[data['target'] == 1][:10000],
                   data[data['target'] == 2][:10000],
                   data[data['target'] == 3][:10000],
                   data[data['target'] == 4][:10000],
                   data[data['target'] == 5][:10000],
                   data[data['target'] == 6][:10000]])

len(again)

70000

In [7]:
df = again.sample(frac=1)
df.head()

,text,target
1877,방역 방해땐 법정최고형…2차 유행 위기에 '정부의 경고',2
53225,曺薰鉉一日 다케미야 覇權 격돌,5
16485,연일 계속되는 추궁에 심신 피로/현철씨 구치소생활 2주째,2
56645,통일후 청사진/신철균교수 통일광복민족회 세미나서 주장,4
223476,<야구소식>OB 이용호 연봉 140% 인상,5


In [8]:
df.to_csv("../data/public_small.csv", index=False)